In [ ]:
!pip install fsspec==2024.9.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
!pip install transformers datasets peft accelerate torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.6 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from datasets import load_dataset

# Load WritingPrompts dataset
dataset = load_dataset("euclaise/writingprompts")
# truncated_dataset = dataset["train"].train_test_split(test_size=0.45, seed=42)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/837 [00:00<?, ?B/s]

(…)-00000-of-00002-105e07cb0d199464.parquet:   0%|          | 0.00/272M [00:00<?, ?B/s]

(…)-00001-of-00002-4fdb982c11056472.parquet:   0%|          | 0.00/272M [00:00<?, ?B/s]

(…)-00000-of-00001-16503b0c26ed00c6.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

(…)-00000-of-00001-137b93e1e979d138.parquet:   0%|          | 0.00/30.7M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/272600 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/15138 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/15620 [00:00<?, ? examples/s]

In [ ]:
print(dataset['train'][0])

{'prompt': "[ WP ] You 've finally managed to discover the secret to immortality . Suddenly , Death appears before you , hands you a business card , and says , `` When you realize living forever sucks , call this number , I 've got a job offer for you . ''\n", 'story': "So many times have I walked on ruins, the remainings of places that I loved and got used to.. At first I was scared, each time I could feel my city, my current generation collapse, break into the black hole that thrives within it, I could feel humanity, the way I'm able to feel my body.. After a few hundred years, the pattern became obvious, no longer the war and damage that would devastate me over and over again in the far past was effecting me so dominantly. \n It's funny, but I felt as if after gaining what I desired so long, what I have lived for my entire life, only then, when I achieved immortality I started truly aging. \n \n 5 world wars have passed, and now they feel like a simple sickeness that would pass by e

In [ ]:
def clean_and_format(example):
    # Remove `[WP]` tag and extra whitespace from prompts
    prompt = example['prompt'].replace('[WP]', '').strip()
    story = example['story'].strip()

    # Combine into a single formatted string
    formatted = f"Prompt: {prompt}\n\nStory: {story}"
    return {"formatted_text": formatted}

# Apply cleaning and formatting
formatted_dataset = dataset.map(clean_and_format)

Map:   0%|          | 0/272600 [00:00<?, ? examples/s]

Map:   0%|          | 0/15138 [00:00<?, ? examples/s]

Map:   0%|          | 0/15620 [00:00<?, ? examples/s]

In [ ]:
import torch
from peft import LoraConfig, get_peft_model, TaskType
from datasets import load_dataset
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [ ]:
# Load the tokenizer for your chosen model
model_name = "Qwen/Qwen2.5-7B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/3.56G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

In [ ]:
pad_token_id = tokenizer.eos_token_id
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
# Modify the tokenization function to include labels
def tokenize_function(examples):
    tokens = tokenizer(
        examples['formatted_text'],
        max_length=1024,
        truncation=True,
        padding="max_length"
    )
    tokens["labels"] = tokens["input_ids"].copy()  # Labels should match input_ids for causal language modeling
    return tokens

tokenized_dataset = formatted_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/272600 [00:00<?, ? examples/s]

Map:   0%|          | 0/15138 [00:00<?, ? examples/s]

Map:   0%|          | 0/15620 [00:00<?, ? examples/s]

In [ ]:
# Prepare LoRA configuration
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=16,                # Low-rank adaptation parameter
    lora_alpha=32,           # Scaling factor
    lora_dropout=0.1,          # Dropout to prevent overfitting
    target_modules=[
        f"layers.{i}.self_attn.q_proj" for i in range(20, 28)
    ] + [
        f"layers.{i}.self_attn.k_proj" for i in range(20, 28)
    ] + [
        f"layers.{i}.self_attn.v_proj" for i in range(20, 28)
    ],
    bias="none"
)

In [ ]:
model = get_peft_model(model, lora_config)

In [ ]:
# Set training arguments
training_args = TrainingArguments(
    output_dir="./qwen-7b-finetuned",
    run_name="qwen-7b-run",
    eval_strategy="no",
    # eval_steps=1000,
    # logging_steps=100,
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=8,
    num_train_epochs=1,
    weight_decay=0.1,
    max_grad_norm=1.0,
    save_steps=200,
    save_total_limit=2,
    logging_dir="./logs-qwen-7b-finetuned",
    logging_strategy="steps",
    fp16=True,
    # remove_unused_columns=False,
    dataloader_num_workers=4
)

tokenized_train = tokenized_dataset["train"]

from transformers import DataCollatorForSeq2Seq
# Define a data collator
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, padding=True)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    data_collator=data_collator
)

# Fine-tune the model with LoRA
trainer.train()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
500,3.316900
1000,1.670100
1500,1.644200
2000,1.645400
2500,1.652000
3000,1.654300
3500,1.652000
4000,1.641700
4500,1.624000
5000,1.640400


TrainOutput(global_step=8518, training_loss=1.7394906232322878, metrics={'train_runtime': 35628.528, 'train_samples_per_second': 7.651, 'train_steps_per_second': 0.239, 'total_flos': 1.184450757326694e+19, 'train_loss': 1.7394906232322878, 'epoch': 0.99991195891416})

In [ ]:
# Define save path
model_save_path = "/content/drive/MyDrive/Creativity/Qwen2.5_7b_fine_tuned_model_euclaise_WritingPrompts"

# Save the fine-tuned model
model.save_pretrained(model_save_path)
tokenizer.save_pretrained(model_save_path)

('/content/drive/MyDrive/Creativity/Qwen2.5_7b_fine_tuned_model_euclaise_WritingPrompts/tokenizer_config.json',
 '/content/drive/MyDrive/Creativity/Qwen2.5_7b_fine_tuned_model_euclaise_WritingPrompts/special_tokens_map.json',
 '/content/drive/MyDrive/Creativity/Qwen2.5_7b_fine_tuned_model_euclaise_WritingPrompts/vocab.json',
 '/content/drive/MyDrive/Creativity/Qwen2.5_7b_fine_tuned_model_euclaise_WritingPrompts/merges.txt',
 '/content/drive/MyDrive/Creativity/Qwen2.5_7b_fine_tuned_model_euclaise_WritingPrompts/added_tokens.json',
 '/content/drive/MyDrive/Creativity/Qwen2.5_7b_fine_tuned_model_euclaise_WritingPrompts/tokenizer.json')